In [ ]:
!pip install konlpy
!pip install wordcloud
!pip install collection
!pip install matplotlib
!apt-get install fonts-nanum*
!apt-get install fontconfig
!pip install WordCloud

!pip install openpylx


In [ ]:
# 모듈 불러오기
import matplotlib
import pandas as pd
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from konlpy.tag import Okt
import re
from collections import Counter
from matplotlib.pyplot import barh


In [ ]:

#unnamed 컬럼이름 'num'으로 바꿔주기
#lunch.rename(columns = {'Unnamed: 0': 'num'}, inplace=True)
def rename_Unnamed(dataFrame):
    dataFrame.rename(columns = {'Unnamed: 0': 'num'}, inplace=True)


In [ ]:

#필터링 하기
def filltering(dataFrame, column_name, string):
    mask = dataFrame[column_name].str.contains(string)
    result = dataFrame[mask]
    return result

In [ ]:

#text 합치기 
def make_all_text(dataFrame, columnName):
    dataFrame.reset_index(drop=True, inplace=True)
    result = ""

    if dataFrame[columnName][0][0] == '[':
        for i in dataFrame.index:
            text = " ".join(dataFrame[columnName][i][1:-1].replace(' ', '').replace('\'', '').split(','))
            result = result + " " + text

    else:
        for i in dataFrame.index:
            text = "".join(dataFrame[columnName][i])
            result = result + " " + text
    return result

In [ ]:

#wordcloud 만들기
#딕셔너리 or 텍스트 넣기
def make_wordcloud(all_text):
    wordcloud = WordCloud(
        #font_path = 'C:\\Windows\\Fonts\\H2GTRE.TTF',
        font_path = '/usr/share/fonts/truetype/nanum/NanumGothic.ttf',
        width = 1600,
        height = 900
    )
    
    if type(all_text) == dict:
      result = wordcloud.generate_from_frequencies(all_text)
    else:
      result = wordcloud.generate(all_text)
    
    plt.imshow(
    result
    #interpolation='bilinear'
    )
    plt.axis("off")
    plt.show()


In [ ]:

#좋아요 콤마 지우고 int로 바꿔주기
#스트링으로 들어온 데이터를 int64로 바꿔준다.
def make_like(dataFrame, columnName):
    dataFrame[columnName] = dataFrame[columnName].str.replace(',', '')
    dataFrame[columnName] = dataFrame[columnName].fillna(0).astype('int64')

    return dataFrame


In [ ]:
#여기서만 사용할 함수?
#태그를 한 리스트로 반환
def make_tags_list(tags_series):
    result = []
    for i in tags_series:
        temp_list = i[2: -2].split("', '")
        result = result + temp_list
    return result


In [ ]:

#tag_value_counts 상위 n개 태그의 빈도수 df 반환
def value_counts_num(listOrSeries, num):
    if type(listOrSeries) == list:
      result = pd.DataFrame(Counter(listOrSeries).most_common(num))
    else:
      temp = make_tags_list(listOrSeries)
      result = pd.DataFrame(Counter(temp).most_common(num))
    return result

In [ ]:
    
#plot 한글 설정
def han_font_set():
    from matplotlib import font_manager, rc
    font_name = font_manager.FontProperties(fname='/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf').get_name()
    matplotlib.rc('font', family = font_name)
    matplotlib.font_manager._rebuild()

In [ ]:

#content 전처리
def prepro_content(dataFrame):
    dataFrame = dataFrame.fillna("nan")
    dataFrame.isnull().sum() # null 사라짐

    def text_cleaner(text):
        clean = text
        publisher = "\((.*?)\)"
        braces = "\[(.*?)\]"
        braces2 = "\{(.*?)\}"
        braces3 = "\【(.*?)\】"
        weird = "[=_\.,;:~…\"\"\'\'◇%\<\>/·○★☆ㅡ]"
        tab = '\\t'
        newline = '\\n'

        clean = re.sub(publisher,'', clean)
        clean = re.sub(braces,'', clean)
        clean = re.sub(braces2,'', clean)
        clean = re.sub(braces3,'', clean)
        clean = re.sub('[YTN,OSEN]','', clean)
        clean = re.sub(weird,'', clean)
        clean = re.sub(tab,'', clean)
        clean = re.sub(newline,'',clean)
        
        return clean

    for idx, text in enumerate(dataFrame['content']):
        dataFrame['content'][idx] = text_cleaner(text)


In [ ]:

#나중에 학습시킬때 사용 
def make_vocab(dataframe):
    okt = Okt()

    sentences_pos = []
    for line in dataframe['content']:
        sentences_pos.append(okt.nouns(line))

    max_len = max([len(i) for i in sentences_pos])

    vocab = []
    for line in sentences_pos:
        for word in line:
            vocab.append(word)

    vocab_size = len(vocab) +1
    vocab = sorted(list(vocab))

   
    vocab = [item for item in vocab if len(item) != 1]

    return sentences_pos, vocab


In [ ]:

#데이터 로드
ato = pd.read_excel('atozzang.xlsx')
bbo = pd.read_excel('bbo.xlsx')
haewon = pd.read_excel('haewon.xlsx')
twomuk = pd.read_excel('2muk.xlsx')
muksta = pd.read_excel('muksta.xlsx')
dfs = [ato, bbo, haewon, twomuk, muksta]

In [ ]:
#content 분석

#content 전처리
#1
for i in dfs:
  prepro_content(i)
#2
sentences_pos = []
vocab = []
for i in dfs:
  tempS, tempV = make_vocab(i)
  sentences_pos.append(tempS)
  vocab.append(tempV)
#3
vocab_value_counts = []
for i in vocab:
  vocab_value_counts.append(value_counts_num(i, 15))



In [ ]:

#태그분석

#태그 전처리
#1
for i in dfs:
    rename_Unnamed(i)

#태그수 보기
tag_value_counts = []
for i in dfs:
    tag_value_counts.append(value_counts_num(i['tags'], 15))


#워드크라우드 용 text파일
text_list = []
for i in dfs:
    text_list.append(make_all_text(i, 'tags'))


In [ ]:

#content 시각화
#barplot
han_font_set()

fig = plt.figure(figsize = (20,20))
fig.add_subplot(2, 3, 1)
barh(vocab_value_counts[0][0], vocab_value_counts[0][1])
fig.add_subplot(2, 3, 2)
barh(vocab_value_counts[1][0], vocab_value_counts[1][1])
fig.add_subplot(2,3,3)
barh(vocab_value_counts[2][0], vocab_value_counts[2][1])
fig.add_subplot(2,3,4)
barh(vocab_value_counts[3][0], vocab_value_counts[3][1])
fig.add_subplot(2,3,5)
barh(vocab_value_counts[4][0], vocab_value_counts[4][1])
plt.show()



In [ ]:


#tag 시각화
#barplot

fig = plt.figure(figsize = (20,20))
fig.add_subplot(2, 3, 1)
barh(tag_value_counts[0][0], tag_value_counts[0][1])
fig.add_subplot(2, 3, 2)
barh(tag_value_counts[1][0], tag_value_counts[1][1])
fig.add_subplot(2,3,3)
barh(tag_value_counts[2][0], tag_value_counts[2][1])
fig.add_subplot(2,3,4)
barh(tag_value_counts[3][0], tag_value_counts[3][1])
fig.add_subplot(2,3,5)
barh(tag_value_counts[4][0], tag_value_counts[4][1])
plt.show()



In [ ]:


#시각화
#word cloud

for i in range(len(vocab_value_counts)):
  temp = vocab_value_counts[i].set_index(0).to_dict()[1]
  make_wordcloud(temp)

#tag 시각화
#word cloud

for i in range(len(tag_value_counts)):
  temp = tag_value_counts[i].set_index(0).to_dict()[1]
  make_wordcloud(temp)
